<a href="https://colab.research.google.com/github/VyomSawhney/Brain-Tumor-MRI-Detection/blob/main/Brain_Cancer_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Brain Cancer Convolutional Neural Network - by Vyom Sawhney

##Importing Kaggle Dataset

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas


# Username and Key can be found in JSON downloaded at https://www.kaggle.com/settings under "Create New Token" button
od.download(
    "https://www.kaggle.com/datasets/orvile/brain-cancer-mri-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vyomsawhney
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/orvile/brain-cancer-mri-dataset


100%|██████████| 144M/144M [00:00<00:00, 471MB/s]


In [ ]:
import os
import shutil

base_path = "/content"

src = os.path.join(base_path, "brain-cancer-mri-dataset", "Brain_Cancer raw MRI data", "Brain_Cancer")
dst = os.path.join(base_path, "brain_cancer")
shutil.move(src, dst)

shutil.rmtree(os.path.join(base_path, "brain-cancer-mri-dataset"))

print("Dataset folder cleaned and ready.")

Dataset folder cleaned and ready.


##Process Data

In [ ]:
import torch
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms

In [ ]:
# Transformation/Augmentations for training dataset
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Resizing but no augmentation for test dataset
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(root="/content/brain_cancer", transform=base_transform)

In [ ]:
# 70% Train, 15% val and 15% Test
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Set transformations
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = base_transform
test_dataset.dataset.transform = base_transform

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

##ResNet18 Transfer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Replace the FC layer allowing to train the model to our 3 classes
num_classes = 3  # Brain_Glioma, Brain_Menin, Brain_Tumor
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 187MB/s]


In [ ]:
# Number of training iteration, 1 iteration is forward->loss->backwards->update
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = train_loss / total
    accuracy = correct / total * 100

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_loss:.4f} | Train Accuracy: {accuracy:.2f}%")


Epoch 1/10 | Train Loss: 0.4872 | Train Accuracy: 81.13%
Epoch 2/10 | Train Loss: 0.1855 | Train Accuracy: 94.72%
Epoch 3/10 | Train Loss: 0.1092 | Train Accuracy: 97.03%
Epoch 4/10 | Train Loss: 0.0641 | Train Accuracy: 98.68%
Epoch 5/10 | Train Loss: 0.0384 | Train Accuracy: 99.29%
Epoch 6/10 | Train Loss: 0.0265 | Train Accuracy: 99.72%
Epoch 7/10 | Train Loss: 0.0161 | Train Accuracy: 99.95%
Epoch 8/10 | Train Loss: 0.0134 | Train Accuracy: 99.93%
Epoch 9/10 | Train Loss: 0.0094 | Train Accuracy: 99.95%
Epoch 10/10 | Train Loss: 0.0085 | Train Accuracy: 99.98%


In [ ]:
def evaluate_model(loader, model):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total * 100
    print(f"Validation Accuracy: {accuracy:.2f}%")

In [ ]:
evaluate_model(val_loader, model)

Validation Accuracy: 98.90%
